In [ ]:
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt
import scipy

# read the data

In [ ]:
#input nino34
with nc.Dataset('/Users/rowenxu/LSTM_EA_Monsoon/Data/input/nino34_uv.nc','r') as f:
    latitude_nino34=f.variables["latitude"][:]
    longitude_nino34=f.variables["longitude"][:]
    time=f.variables["time"][:]
    uwind_nino34=f.variables["u"][:]
    vwind_nino34=f.variables["v"][:]
#input nino12
with nc.Dataset('/Users/rowenxu/LSTM_EA_Monsoon/Data/input/nino12_uv.nc','r') as f:
    latitude_nino12=f.variables["latitude"][:]
    longitude_nino12=f.variables["longitude"][:]
    time=f.variables["time"][:]
    uwind_nino12=f.variables["u"][:]
    vwind_nino12=f.variables["v"][:]
#input the sst
with nc.Dataset('/Users/rowenxu/LSTM_EA_Monsoon/Data/input/nino1234_sst.nc','r') as f:
    sst_nino1234=f.variables["t"][:]
    latitude_1234=f.variables["latitude"][:]
    longitude_1234=f.variables["longitude"][:]
    index_lat_nino12=[i for i in range(len(latitude_1234)) if latitude_1234[i]>=-10 and latitude_1234[i]<=0]
    index_lon_nino12=[i for i in range(len(longitude_1234)) if longitude_1234[i]>=-90 and longitude_1234[i]<=-80]
    index_lat_nino34=[i for i in range(len(latitude_1234)) if latitude_1234[i]>=-5 and latitude_1234[i]<=5]
    index_lon_nino34=[i for i in range(len(longitude_1234)) if longitude_1234[i]>=-170 and longitude_1234[i]<=-120]
    sst_nino34=sst_nino1234[:,0,index_lat_nino34[0]:index_lat_nino34[-1],index_lon_nino34[0]:index_lon_nino34[-1]]#nino34 is 5s to 5n in lat and 170W-120W in lon
    sst_nino12=sst_nino1234[:,0,index_lat_nino12[0]:index_lat_nino12[-1],index_lon_nino12[0]:index_lon_nino12[-1]]#nino12 is 10S-0N in lat and 90W-80W in lon

In [ ]:
print(sst_nino34.shape)
print(sst_nino12.shape)
#对nino34点经纬度进行下采样，使得与12区的分辨率一致
sst_nino34_downsample=np.zeros((sst_nino34.shape[0],sst_nino12.shape[1],sst_nino12.shape[2]))
for i in range(sst_nino34.shape[0]):
    sst_nino34_downsample[i,:,:]=scipy.ndimage.zoom(sst_nino34[i,:,:],(1,2,2),order=1)


In [ ]:
print(sst_nino34_downsample.shape)

In [ ]:
#normalize the data
uwind_norm_nino34=np.zeros(uwind_nino34.shape)
vwind_norm_nino34=np.zeros(vwind_nino34.shape)
uwind_norm_nono12=np.zeros(uwind_nino12.shape)#normalize the data
uwind_norm_nino34=np.zeros(uwind_nino34.shape)
vwind_norm_nino34=np.zeros(vwind_nino34.shape)
uwind_norm_nono12=np.zeros((len(time),len(longitude_nino12),len(latitude_nino12)))
for z in range(2):
    for i in range(len(latitude_nino34)):
        for j in range(len(longitude_nino34)):
            uwind_norm_nino34[:,z,i,j]=(uwind_nino34[:,z,i,j]-np.mean(uwind_nino34[:,z,i,j]))/np.std(uwind_nino34[:,z,i,j])
            vwind_norm_nino34[:,z,i,j]=(vwind_nino34[:,z,i,j]-np.mean(vwind_nino34[:,z,i,j]))/np.std(vwind_nino34[:,z,i,j])
for z in range(2):
    for i in range(len(latitude_nino34)):
        for j in range(len(longitude_nino34)):
            uwind_norm_nino34[:,z,i,j]=(uwind_nino34[:,z,i,j]-np.mean(uwind_nino34[:,z,i,j]))/np.std(uwind_nino34[:,z,i,j])
            vwind_norm_nino34[:,z,i,j]=(vwind_nino34[:,z,i,j]-np.mean(vwind_nino34[:,z,i,j]))/np.std(vwind_nino34[:,z,i,j])
#calculate the circulation index from the wind
circulation_nino34=np.zeros((len(time),len(latitude_nino34),len(longitude_nino34)))
for i in range(len(latitude_nino34)):
    for j in range(len(longitude_nino34)):
        circulation_nino34[:,i,j]=uwind_norm_nino34[:,0,i,j]*vwind_norm_nino34[:,0,i,j]-uwind_norm_nino34[:,0,i,j]*vwind_norm_nino34[:,0,i,j]




# Calculate the NINO INDEX (1+2) and (3.4)

In [ ]:
nino12_index=np.zeros(len(time))
for i in range(len(time)):
    nino12_index[i]=np.mean(sst_nino12[i,:,:])-np.mean(sst_nino12[:,:,:]) #calculate the nino12 index
fig1=plt.figure()
ax=fig1.add_subplot()
nino12_index_flatten=np.convolve(nino12_index[300:],np.ones(8)/8)
ax.plot(range(len(nino12_index[300:])+7),nino12_index_flatten)

ax.set_xlabel('time series(month))')
ax.set_ylabel('nino12 index(from )')
ax.set_title('index')
nino34_index=np.zeros(len(time))
for i in range(len(time)):
    nino34_index[i]=np.mean(sst_nino34[i,:,:])-np.mean(sst_nino34[:,:,:]) #calculate the nino34 index
nino34_index_flatten=np.convolve(nino34_index[300:],np.ones(8)/8)
ax.plot(range(len(nino34_index[300:])+7),nino34_index_flatten)
ax.set_xlabel('time series(month))')
ax.set_ylabel('index')
corr=np.corrcoef(nino12_index[300:500],nino34_index[300:500])
ax.legend(['nino12','nino34'])
# ax.text(100,100,'corrcoef='+str(corr[0,1]))
#since 2020.5, the longest lanina event continues until 2023.3 as a result, we need to hightlight the curve in this event
